In [1]:
import cv2
import os
from vidaug import augmentors as va
import numpy
import skvideo.io
import pandas as pd
import numpy as np
import imageio

In [2]:
# >Iterate DAISEEsplitted/0 and save the name of each video in a dataframe
df = pd.DataFrame(columns=['ClipID', 'label'])
for i in range(0, 2):
    path = 'DAISEEsplitted/' + str(i) + '/'
    for filename in os.listdir(path):
        # if end with .avi or .mp4, append to dataframe
        if filename.endswith('.avi') or filename.endswith('.mp4'):
            new_row = pd.DataFrame({'ClipID': [filename], 'label': [i]})
            df = pd.concat([df, new_row], ignore_index=True)

In [3]:
# Divide the dataframe into train for label 0 and label 1
train_0 = df[df['label'] == 0].sample(frac=0.75, random_state=123)
train_1 = df[df['label'] == 1].sample(frac=0.66, random_state=123)
# get the rest of the data for validation
val_0 = df[df['label'] == 0].drop(train_0.index)
val_1 = df[df['label'] == 1].drop(train_1.index)

train_0.shape, train_1.shape, val_0.shape, val_1.shape

((46, 2), (300, 2), (15, 2), (155, 2))

In [4]:
augClass0Path = "AugDaisee/Batches/batch_01/0/"
augClass1Path = "AugDaisee/Batches/batch_01/1/"

In [5]:
def frames_extraction(video_path):
    frames_list = []
    seqCount = 1
    vidObj = cv2.VideoCapture(video_path)
    while seqCount <= 300:
        success, image = vidObj.read()
        if success:
            frames_list.append(image)
            seqCount += 1
        else:
            print("Defected frame")
            break
    vidObj.release()
    return frames_list

In [10]:
#1. Apply class0 aug, one by one till reach 276, 552 and finally 1,104
#2. Change classpath to be class1Path, instead of class0Path
#3. Aug class1 to reach 600 video, then to reach 1,200 video
#NB: at every aug check that right aug and augclass are uncommented + classPath, augClassPath and file name are selected correctly

In [ ]:
# Probabilidad para algunos augmentos (si quieres usar sometimes)
sometimes = lambda aug: va.Sometimes(0.5, aug)

# Define múltiples combinaciones de augmentación (puedes poner 15 o más)
combinations = [
    #[va.Salt()],
    #[va.Add(-20)],
    #[va.Pepper()],
    [va.Multiply(2)],
    [va.GaussianBlur(0.9)],
    [va.InvertColor()],
    [va.ElasticTransformation(alpha=5, sigma=1)],
    [va.Salt(), va.Add(-20)],
    [va.Pepper(), va.Multiply(2)],
    [va.GaussianBlur(0.9), va.InvertColor()],
    [va.ElasticTransformation(alpha=5, sigma=1), va.Salt()],
    [sometimes(va.HorizontalFlip())],
    [va.RandomShear(0.2, 0.1)],
    [va.Add(-20), va.Multiply(2)],
    [va.Salt(), va.GaussianBlur(0.9)]
]

#combinations = [
#    [va.RandomShear(0.2,0.1)],
#    [va.RandomShear(0.2,0.1), sometimes(va.HorizontalFlip())],
#    [va.RandomShear(0.2,0.1), sometimes(va.HorizontalFlip()), va.RandomShear(0.2,0.1), sometimes(va.HorizontalFlip())],
#]
    
def video_augmentation(augClassPath, train):
    videos = train['ClipID'].tolist()
    pathSplitted = 'DAISEEsplitted/' + augClassPath[-2]

    for v in videos:
        video = frames_extraction(os.path.join(pathSplitted, v))

        # Guardar video original sin cambios (si no existe)
        save_name = f"aug0000-{v}.mp4"
        savedVideo = os.path.join(augClassPath, save_name)

        if not os.path.exists(savedVideo):
            with imageio.get_writer(savedVideo, fps=30, format='ffmpeg') as writer:
                for frame in video:
                    writer.append_data(frame)
            print("Original guardado:", savedVideo)
        else:
            print("Original ya existía:", savedVideo)

        # Aplicar cada combinación
        for idx, combo in enumerate(combinations):
            seq = va.Sequential(combo)
            video_aug = seq(video)
            video_aug = np.array(video_aug).astype(np.uint8)

            save_name = f"aug{idx+1:04d}-{v}.mp4"  # aug01-, aug02-, ...
            savedVideo = os.path.join(augClassPath, save_name)

            with imageio.get_writer(savedVideo, fps=30, format='ffmpeg') as writer:
                for frame in video_aug:
                    writer.append_data(frame)

            print("Augmentado guardado:", savedVideo)

# Ejecutar para class 0
video_augmentation(augClass0Path, train_0)

Original guardado: AugDaisee/Batches/batch_01/1/aug0000-2000501030.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0001-2000501030.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0002-2000501030.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0003-2000501030.avi.mp4
Original guardado: AugDaisee/Batches/batch_01/1/aug0000-2100591046.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0001-2100591046.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0002-2100591046.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0003-2100591046.avi.mp4
Original guardado: AugDaisee/Batches/batch_01/1/aug0000-1100172043.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0001-1100172043.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0002-1100172043.avi.mp4
Augmentado guardado: AugDaisee/Batches/batch_01/1/aug0003-1100172043.avi.mp4
Original guardado: AugDaisee/Batches/batch_01/1/aug0000-4100251036.avi.mp4
Augment

In [ ]:
augClass0Path

In [ ]:
train_0['ClipID'].tolist()[0][:5]